In [0]:
import requests
import os
from requests.packages.urllib3.exceptions import InsecureRequestWarning

In [0]:
url_estados = "https://raw.githubusercontent.com/horstao/municipios-brasileiros-completo/master/csv/estados.csv"
url_municipios = "https://raw.githubusercontent.com/horstao/municipios-brasileiros-completo/master/csv/municipios.csv"

In [0]:
diretorio_destino = "/Volumes/controlled_medication_data/bronze_layer/raw_data"
os.makedirs(diretorio_destino, exist_ok=True)

In [0]:


# Diretorio onde os arquivos serão salvos
diretorio_destino = "/Volumes/controlled_medication_data/bronze_layer/raw_data"
os.makedirs(diretorio_destino, exist_ok=True)

# Lista de arquivos para download (nome do arquivo, URL)
arquivos_para_baixar = [
    ("estados.csv", url_estados),
    ("municipios.csv", url_municipios)
]

for nome_arquivo, url in arquivos_para_baixar:
    caminho_completo = os.path.join(diretorio_destino, nome_arquivo)
    
    print(f"Baixando {nome_arquivo}...")
    try:
        response = requests.get(url)
        # Verifica se a requisição foi bem sucedida
        response.raise_for_status()

        with open(caminho_completo, "wb") as f:
            f.write(response.content)
        
        print(f"{nome_arquivo} salvo com sucesso em '{caminho_completo}'")
    except requests.exceptions.RequestException as e:
        print(f"Erro ao baixar {nome_arquivo}: {e}")

print("---")
print("Processo de download finalizado.")

In [0]:
df_estados = spark.read.csv("/Volumes/controlled_medication_data/bronze_layer/raw_data/estados.csv", header=True, inferSchema=True)

df_estados.display()

In [0]:
df_estados = df_estados.withColumnRenamed("nome", "nome_uf")
df_estados = df_estados.withColumnRenamed("latitude", "latitude_uf")
df_estados = df_estados.withColumnRenamed("longitude", "longitude_uf")
df_estados.show(5)

In [0]:
df_municipios = spark.read.csv("/Volumes/controlled_medication_data/bronze_layer/raw_data/municipios.csv", header=True, inferSchema=True)

df_municipios.display()

In [0]:
df_municipios = df_municipios.withColumnRenamed("nome", "nome_municipio")
df_municipios = df_municipios.withColumnRenamed("latitude", "latitude_municipio")
df_municipios = df_municipios.withColumnRenamed("longitude", "longitude_municipio")
df_municipios = df_municipios.drop("siafi_id", "codigo_ibge")
df_municipios.show(5)

In [0]:
df_resultado = df_municipios.join(df_estados, on="codigo_uf", how="inner")

In [0]:
df_resultado.display()

In [0]:
from pyspark.sql.functions import lower, upper, col

df_resultado = df_resultado.withColumn("nome_municipio", upper(col("nome_municipio")))

In [0]:
df_resultado.display()

In [0]:
df_resultado.write.mode("overwrite").saveAsTable("controlled_medication_data.bronze_layer.municipios")

In [0]:
%sql

SELECT * 
FROM controlled_medication_data.bronze_layer.municipios
WHERE nome_municipio = "SÃO PAULO" AND uf = "SP"